In [ ]:
import pandas as pd

df_train = pd.read_csv('Dataset/conjunto_de_treinamento.csv')

In [ ]:
def DataProcessing1(df):
    df.drop(columns=['Id'], inplace=True)
    df.drop(columns=['diferenciais'], inplace=True)

    df['tipo'] = df['tipo'].map({'Apartamento': 1, 'Casa': 2})
    df['tipo'].fillna(0, inplace=True)
    df['tipo_vendedor'] = df['tipo_vendedor'].map({'Imobiliaria': 0, 'Pessoa Fisica': 1})

In [ ]:
def DataProcessing2(df):
    # Calcula a média dos valores dos imóveis por bairro
    mean_encoded = df.groupby('bairro')['preco'].mean()

    # Mapeia os valores médios dos imóveis por bairro para os respectivos bairros
    df['Bairro_Encoded'] = df['bairro'].map(mean_encoded)

    # Obtém a lista de todas as colunas no DataFrame
    colunas = df.columns.tolist()

    # Move a coluna 'Bairro_Encoded' para a posição desejada, logo após a coluna 'Bairro'
    colunas.insert(colunas.index('bairro') + 1, colunas.pop(colunas.index('Bairro_Encoded')))

    # Reindexa o DataFrame com as colunas na nova ordem
    df = df.reindex(columns=colunas)
    df.drop(columns=['bairro'], inplace=True)
    return df


In [ ]:
# # Verifica se a coluna 'tipo' contém valores NaN
# df_train['tipo'].fillna(-1, inplace=True)
# tem_nan = df_train['tipo'].isna().any()
# print(df_train['tipo'].value_counts()
# )
# # Exibe o resultado
# tem_nan

In [ ]:
df_test = pd.read_csv('Dataset/conjunto_de_teste.csv')

DataProcessing1(df_train)
DataProcessing1(df_test)
df_train = DataProcessing2(df_train)


In [ ]:

df_test['bairro'] = df_train['Bairro_Encoded']

df_test.head()


In [ ]:
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values

X_test = df_test.iloc[:, :].values
# y_test = df_test.iloc[:, -1].values

X_train.shape, X_test.shape


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [ ]:
# Criar DataFrame com a coluna 'Id' e 'preco' contendo os valores de y_pred
df_resultado = pd.DataFrame({'Id': range(len(y_pred)), 'preco': y_pred})

# Salvar o DataFrame em um arquivo CSV
df_resultado.to_csv('resultado.csv', index=False)